# 仅流式传输最终代理输出

如果您只想流式传输代理的最终输出，则可以使用回调 ``FinalStreamingStdOutCallbackHandler``。
为此，基础的 LLM 也必须支持流式传输。

In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler
from langchain.llms import OpenAI

让我们使用 ``streaming=True`` 创建基础的 LLM，并传递 ``FinalStreamingStdOutCallbackHandler`` 的一个新实例。

In [2]:
llm = OpenAI(streaming=True, callbacks=[FinalStreamingStdOutCallbackHandler()], temperature=0)

In [4]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False)
agent.run("It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany.")

 Konrad Adenauer became Chancellor of Germany in 1949, 74 years ago in 2023.

'Konrad Adenauer became Chancellor of Germany in 1949, 74 years ago in 2023.'

### 处理自定义答案前缀

默认情况下，我们假定标记序列 ``"Final", "Answer", ":"`` 意味着代理已经得到了一个答案。但是，我们也可以传递自定义序列作为答案前缀。

In [5]:
llm = OpenAI(
    streaming=True,
    callbacks=[FinalStreamingStdOutCallbackHandler(answer_prefix_tokens=["The", "answer", ":"])],
    temperature=0
)

为了方便起见，当与 `answer_prefix_tokens` 进行比较时，回调会自动去除空格和换行符。也就是说，如果 `answer_prefix_tokens = ["The", " answer", ":"]`，则 `["\nThe", " answer", ":"]` 和 `["The", " answer", ":"]` 都会被认为是答案前缀。

如果您不知道答案前缀的标记化版本，可以使用以下代码确定：

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler

class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs) -> None:
        # print every token on a new line
        print(f"#{token}#")

llm = OpenAI(streaming=True, callbacks=[MyCallbackHandler()])
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False)
agent.run("It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany.")

### 同时流式传输答案前缀

当参数 `stream_prefix=True` 时，答案前缀本身也将被流式传输。当答案前缀本身是答案的一部分时，这可能很有用。例如，当您的答案是类似于以下 JSON 格式时：

`
{
    "action": "Final answer",
    "action_input": "Konrad Adenauer became Chancellor 74 years ago."
}
`

您不仅想要流式传输 action_input，而是整个 JSON。